In [1]:
import pandas as pd
import numpy as np
import gc

origin_train = pd.read_csv('./data/train.csv')
origin_sup = pd.read_csv('./data/supplemental_train.csv')

- missing value in `VWAP`,`Target` at some timestamp.
- missing timestamp for some assets.

In [2]:
#show time range
time_range=[]
for i in range(14):
    range_digit=origin_train.loc[origin_train['Asset_ID']==i,'timestamp'].agg(['min', 'max'])
    range_digit.name = f'Asset_ID_{i}'
    time_range.append(pd.DataFrame([range_digit]))
tt=pd.concat(time_range)
tt['min_date'] = pd.to_datetime(tt['min'],unit="s",infer_datetime_format=True)
tt['max_date'] = pd.to_datetime(tt['max'],unit="s",infer_datetime_format=True)
tt

,min,max,min_date,max_date
Asset_ID_0,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_1,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_2,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_3,1523956260,1632182400,2018-04-17 09:11:00,2021-09-21
Asset_ID_4,1555079640,1632182400,2019-04-12 14:34:00,2021-09-21
Asset_ID_5,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_6,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_7,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21
Asset_ID_8,1525853220,1632182400,2018-05-09 08:07:00,2021-09-21
Asset_ID_9,1514764860,1632182400,2018-01-01 00:01:00,2021-09-21


In [3]:
#show time range
time_range=[]
for i in range(14):
    range_digit=origin_sup.loc[origin_sup['Asset_ID']==i,'timestamp'].agg(['min', 'max'])
    range_digit.name = f'Asset_ID_{i}'
    time_range.append(pd.DataFrame([range_digit]))
tt=pd.concat(time_range)
tt['min_date'] = pd.to_datetime(tt['min'],unit="s",infer_datetime_format=True)
tt['max_date'] = pd.to_datetime(tt['max'],unit="s",infer_datetime_format=True)
tt

,min,max,min_date,max_date
Asset_ID_0,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_1,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_2,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_3,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_4,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_5,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_6,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_7,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_8,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24
Asset_ID_9,1632182460,1642982400,2021-09-21 00:01:00,2022-01-24


In [4]:
train = pd.concat([origin_train,origin_sup])
del origin_train, origin_sup
gc.collect()
print('data loaded')
#show time range
time_range=[]
for i in range(14):
    range_digit=train.loc[train['Asset_ID']==i,'timestamp'].agg(['min', 'max'])
    range_digit.name = f'Asset_ID_{i}'
    time_range.append(pd.DataFrame([range_digit]))
tt=pd.concat(time_range)
tt['min_date'] = pd.to_datetime(tt['min'],unit="s",infer_datetime_format=True)
tt['max_date'] = pd.to_datetime(tt['max'],unit="s",infer_datetime_format=True)
tt

data loaded


,min,max,min_date,max_date
Asset_ID_0,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_1,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_2,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_3,1523956260,1642982400,2018-04-17 09:11:00,2022-01-24
Asset_ID_4,1555079640,1642982400,2019-04-12 14:34:00,2022-01-24
Asset_ID_5,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_6,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_7,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24
Asset_ID_8,1525853220,1642982400,2018-05-09 08:07:00,2022-01-24
Asset_ID_9,1514764860,1642982400,2018-01-01 00:01:00,2022-01-24


In [5]:
train = train.set_index("timestamp").sort_index()
#add normalized weight
asset_detail = pd.read_csv('./data/asset_details.csv')
weight_dict=dict(zip(asset_detail.Asset_ID, asset_detail.Weight/asset_detail.Weight.sum()))
train['Weight'] = train['Asset_ID'].map(weight_dict)
train.set_index('Asset_ID',append=True, inplace=True)

In [6]:
#######################################add lr_15,mkt_lr_15,crypto_index
def log_return(series, periods=5):
    return np.log(series).diff(periods)

lr_15 = train.groupby('Asset_ID').apply( 
        lambda x: log_return(x[['Close']],15)
        )
train['lr_15'] = lr_15['Close']#same order:[timestamp, asset_id]
del lr_15
gc.collect()
print('lr_15 added')

mkt_lr_15 = train.groupby('timestamp').apply( 
    lambda x: x[["lr_15", "Close"]].multiply(x["Weight"], axis="index").sum()
    )

mkt_lr_15.columns = ['Mkt_lrt_15','Crypto_Index']#single index timestamp 
firsts = train.index.get_level_values('timestamp')
train[['Mkt_lrt_15','Crypto_Index']] = mkt_lr_15.loc[firsts].values
del mkt_lr_15
gc.collect()
print('Mkt_lrt_15,Crypto_Index added')

lr_15 added
Mkt_lrt_15,Crypto_Index added


In [9]:
train

Count         Open          High           Low  \
timestamp  Asset_ID                                                   
1514764860 2          40.0   2376.58000   2399.500000   2357.140000   
           0           5.0      8.53000      8.530000      8.530000   
           1         229.0  13835.19400  14013.800000  13666.110000   
           5          32.0      7.65960      7.659600      7.656700   
           7           5.0     25.92000     25.920000     25.874000   
...                    ...          ...           ...           ...   
1642982400 9         214.0    112.29300    112.470000    111.840000   
           10         25.0   1833.56500   1834.700000   1831.050000   
           13        235.0      0.05752      0.057618      0.057389   
           12        492.0      0.20094      0.201066      0.199352   
           11         40.0    156.76400    156.930000    156.200000   

                            Close        Volume          VWAP    Target  \
timestamp  Asset_ID                                                       
1514764860 2          2374.590000  1.923301e+01   2373.116392 -0.004218   
           0             8.530000  7.838000e+01      8.530000 -0.014399   
           1         13850.176000  3.155006e+01  13827.062093 -0.014643   
           5             7.657600  6.626713e+03      7.657713 -0.013922   
           7            25.877000  1.210873e+02     25.891363 -0.008264   
...                           ...           ...           ...       ...   
1642982400 9           111.983333  1.243860e+03    112.127619       NaN   
           10         1833.018333  4.190290e+00   1833.173692       NaN   
           13            0.057478  8.988253e+05      0.057519       NaN   
           12            0.200093  2.983796e+06      0.200413       NaN   
           11          156.550000  7.439747e+01    156.638720       NaN   

                       Weight     lr_15  Mkt_lrt_15  Crypto_Index  
timestamp  Asset_ID                                                
1514764860 2         0.058657       NaN    0.000000   2571.477256  
           0         0.105286       NaN    0.000000   2571.477256  
           1         0.165850       NaN    0.000000   2571.477256  
           5         0.033911       NaN    0.000000   2571.477256  
           7         0.050867       NaN    0.000000   2571.477256  
...                       ...       ...         ...           ...  
1642982400 9         0.058657 -0.001254   -0.001114   6494.648269  
           10        0.026874 -0.002008   -0.001114   6494.648269  
           13        0.043830 -0.000622   -0.001114   6494.648269  
           12        0.050867 -0.002306   -0.001114   6494.648269  
           11        0.039370 -0.000843   -0.001114   6494.648269  

[26755084 rows x 12 columns]

In [10]:
##fill in missing Target
from script.data_target import *
train = make_target(train)
train['Target'].fillna(train.Target2,inplace=True)
print('filling missing Target')

filling missing Target


In [11]:
train

Count         Open          High           Low  \
timestamp  Asset_ID                                                   
1514764860 2          40.0   2376.58000   2399.500000   2357.140000   
           0           5.0      8.53000      8.530000      8.530000   
           1         229.0  13835.19400  14013.800000  13666.110000   
           5          32.0      7.65960      7.659600      7.656700   
           7           5.0     25.92000     25.920000     25.874000   
...                    ...          ...           ...           ...   
1642982400 9         214.0    112.29300    112.470000    111.840000   
           10         25.0   1833.56500   1834.700000   1831.050000   
           13        235.0      0.05752      0.057618      0.057389   
           12        492.0      0.20094      0.201066      0.199352   
           11         40.0    156.76400    156.930000    156.200000   

                            Close        Volume          VWAP    Target  \
timestamp  Asset_ID                                                       
1514764860 2          2374.590000  1.923301e+01   2373.116392 -0.004218   
           0             8.530000  7.838000e+01      8.530000 -0.014399   
           1         13850.176000  3.155006e+01  13827.062093 -0.014643   
           5             7.657600  6.626713e+03      7.657713 -0.013922   
           7            25.877000  1.210873e+02     25.891363 -0.008264   
...                           ...           ...           ...       ...   
1642982400 9           111.983333  1.243860e+03    112.127619       NaN   
           10         1833.018333  4.190290e+00   1833.173692       NaN   
           13            0.057478  8.988253e+05      0.057519       NaN   
           12            0.200093  2.983796e+06      0.200413       NaN   
           11          156.550000  7.439747e+01    156.638720       NaN   

                       Weight     lr_15  Mkt_lrt_15  Crypto_Index      R_15  \
timestamp  Asset_ID                                                           
1514764860 2         0.058657       NaN    0.000000   2571.477256 -0.004227   
           0         0.105286       NaN    0.000000   2571.477256 -0.014504   
           1         0.165850       NaN    0.000000   2571.477256 -0.014751   
           5         0.033911       NaN    0.000000   2571.477256 -0.014020   
           7         0.050867       NaN    0.000000   2571.477256 -0.008298   
...                       ...       ...         ...           ...       ...   
1642982400 9         0.058657 -0.001254   -0.001114   6494.648269       NaN   
           10        0.026874 -0.002008   -0.001114   6494.648269       NaN   
           13        0.043830 -0.000622   -0.001114   6494.648269       NaN   
           12        0.050867 -0.002306   -0.001114   6494.648269       NaN   
           11        0.039370 -0.000843   -0.001114   6494.648269       NaN   

                      Mkt_lrt  beta   Target2  
timestamp  Asset_ID                            
1514764860 2        -0.006391   0.0 -0.004227  
           0        -0.006391   0.0 -0.014504  
           1        -0.006391   0.0 -0.014751  
           5        -0.006391   0.0 -0.014020  
           7        -0.006391   0.0 -0.008298  
...                       ...   ...       ...  
1642982400 9              NaN   0.0       NaN  
           10             NaN   0.0       NaN  
           13             NaN   0.0       NaN  
           12             NaN   0.0       NaN  
           11             NaN   0.0       NaN  

[26755084 rows x 16 columns]

In [12]:
train.reset_index().drop(['R_15','Mkt_lrt','beta','Target2'],axis=1).to_feather('./data/new_data3.ftr')

In [13]:
print(train.loc[train.Target.isin([np.nan, np.inf, -np.inf])])

                      Count          Open          High           Low  \
timestamp  Asset_ID                                                     
1642981500 3         1082.0      1.118946      1.125600      1.117000   
           2          106.0    301.228000    302.260000    301.000000   
           0          533.0    382.678000    384.040000    382.400000   
           1         2779.0  36227.752857  36356.000000  36196.770000   
           4          536.0      0.140960      0.141720      0.140800   
...                     ...           ...           ...           ...   
1642982400 9          214.0    112.293000    112.470000    111.840000   
           10          25.0   1833.565000   1834.700000   1831.050000   
           13         235.0      0.057520      0.057618      0.057389   
           12         492.0      0.200940      0.201066      0.199352   
           11          40.0    156.764000    156.930000    156.200000   

                            Close        Volume   